### DCGAN by keras
References
    1. https://blog.csdn.net/u010159842/article/details/79042195
    2. https://github.com/myinxd/keras-dcgan

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from keras import Input
from keras import applications

/home/mzx/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math

In [4]:
def generator_model():
    model = Sequential()
    model.add(Dense(units=1024, input_dim=100))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model

In [5]:
def discriminator_model():
    model = Sequential()
    model.add(Conv2D(64, (5, 5),
                     padding='same',
                     input_shape=(28, 28, 1)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [6]:
def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    return model

In [7]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[ :, :, 0]
    return image

In [8]:
def train(BATCH_SIZE):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)
    d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    d_loss_avg = np.zeros(100)
    d_loss_std = np.zeros(100)
    g_loss_avg = np.zeros(100)
    g_loss_std = np.zeros(100)
    for epoch in range(100):
        print("Epoch is", epoch)
        numbatch = int(X_train.shape[0]/BATCH_SIZE)
        d_loss_epoch = np.zeros(numbatch)
        g_loss_epoch = np.zeros(numbatch)
        print("Number of batches", numbatch)
        for index in range(numbatch):
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            if index % 100 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save("results/"+
                    str(epoch)+"_"+str(index)+".png")
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            d_loss_epoch[index] = d_loss
            #print("batch %d d_loss : %f" % (index, d_loss))
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            g_loss_epoch[index] = g_loss
            d.trainable = True
            #print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)
        print("d_loss_avg: %7.5f, g_loss_avg: %7.5f" % (np.mean(d_loss_epoch), np.mean(g_loss_epoch)))
        d_loss_avg[epoch] = np.mean(d_loss_epoch)
        d_loss_std[epoch] = np.std(d_loss_epoch)
        g_loss_avg[epoch] = np.mean(g_loss_epoch)
        g_loss_std[epoch] = np.std(g_loss_epoch)

In [9]:
def generate(BATCH_SIZE, nice=False):
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
        d = discriminator_model()
        d.compile(loss='binary_crossentropy', optimizer="SGD")
        d.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100))
        generated_images = g.predict(noise, verbose=1)
        d_pret = d.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        generated_images = g.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")

In [ ]:
class args(object):
    mode_train="train"
    mode_generate = "generate"
    batch_size=128
    nice="nice"


train(BATCH_SIZE=args.batch_size)
# generate(BATCH_SIZE=args.batch_size, nice=args.nice)

Epoch is 0
Number of batches 468
d_loss_avg: 0.43149, g_loss_avg: 1.12933
Epoch is 1
Number of batches 468
d_loss_avg: 0.45554, g_loss_avg: 1.20593
Epoch is 2
Number of batches 468
d_loss_avg: 0.41342, g_loss_avg: 1.28918
Epoch is 3
Number of batches 468
d_loss_avg: 0.40420, g_loss_avg: 1.39928
Epoch is 4
Number of batches 468
d_loss_avg: 0.27928, g_loss_avg: 1.98459
Epoch is 5
Number of batches 468
d_loss_avg: 0.21729, g_loss_avg: 2.42318
Epoch is 6
Number of batches 468
d_loss_avg: 0.20206, g_loss_avg: 2.63970
Epoch is 7
Number of batches 468
d_loss_avg: 0.26369, g_loss_avg: 2.51940
Epoch is 8
Number of batches 468
d_loss_avg: 0.34619, g_loss_avg: 2.18217
Epoch is 9
Number of batches 468
d_loss_avg: 0.37493, g_loss_avg: 1.97712
Epoch is 10
Number of batches 468
d_loss_avg: 0.38160, g_loss_avg: 1.78985
Epoch is 11
Number of batches 468
d_loss_avg: 0.36916, g_loss_avg: 1.73278
Epoch is 12
Number of batches 468
d_loss_avg: 0.37387, g_loss_avg: 1.78821
Epoch is 13
Number of batches 468
d